In [10]:
import os
from os import listdir
from os.path import isfile, join
import networkx as nx
import numpy as np
import torch_geometric
from torch_geometric.utils import from_networkx
import torch
import re


##  Set the gset folder path as you prefer

In [11]:
#Directory stuff
current_directory = os.getcwd()
gset_directory = "GSET"
gset_path = current_directory+"/"+gset_directory

In [12]:
gset_path

'/home/stalence/repositories/GSET'

In [13]:
#function for reading the gset files
def load_mtx(path):
    with open(path, 'r') as f:
        g = nx.Graph()
        weights = []
        first_line = True
        for line in f:
            if not line[0] == '%':
                s = line.split()
                if first_line:
                    g.add_nodes_from(range(int(s[0])))
                    first_line = False
                else:
                    g.add_edge(int(s[0]) - 1, int(s[1]) - 1)
                    if len(s) > 2:
                        weights.append(int(s[2]))
    if len(weights) < g.number_of_edges():
        weights = None
    else:
        weights = np.int64(weights)
    return g, weights


In [14]:
#Prepping the dataset
graphs_and_weights = {}
optimal_values = []
counter = 0
notgs = 0
gs = []
for file in listdir(gset_path):
    tokens = file.split('.')    
    if len(tokens)==2 and 'mtx' in tokens[1]:
        counter+=1
        g, weights = load_mtx(gset_path+ '/'+file)
        graphs_and_weights[tokens[0]]=[g,weights]
    for token in tokens:
        if 'txt' in token:
            with open(gset_path+ '/'+file, mode='r',  encoding='utf-8-sig') as f:
                lines = f.readlines()
                for chunk in lines:
                    splitline = chunk.split('\t')
                    if not 'G' in chunk:
                        notgs+=1
                        optimal = splitline[1].split('(')[0]
                        optimal = optimal.replace(".","")
                        optimal = optimal.replace(",","")
                        optimal_values+=[optimal]
                    else:
                        notgs+=1
                        currg = splitline[0]
                        gs+=[currg]
                    parts = re.split('\n | \t', chunk)
                    


In [15]:
best_known_gs = {}
for g,opt in zip(gs,optimal_values):
    best_known_gs[g]=opt

In [16]:
for g in graphs_and_weights.keys():
    graphs_and_weights[g]+=[best_known_gs[g]]

In [17]:
pyg_dataset = []
for graph in graphs_and_weights.keys():
    pyg_graph = from_networkx(graphs_and_weights[graph][0])
    if graphs_and_weights[graph][1] is None:
        pyg_graph.weights = torch.ones(graphs_and_weights[graph][0].number_of_nodes(),graphs_and_weights[graph][0].number_of_nodes())
    else:
        pyg_graph.weights = torch.FloatTensor(graphs_and_weights[graph][1])
        print(graphs_and_weights[graph][2])
    pyg_graph.optimal = torch.FloatTensor(int(graphs_and_weights[graph][2]))
    pyg_graph.name = graph
    pyg_dataset+=[pyg_graph]
    
        

9541


In [18]:
pyg_dataset

[Data(edge_index=[2, 11832], num_nodes=1000, weights=[1000, 1000], optimal=[3852], name='G54'),
 Data(edge_index=[2, 9322], num_nodes=800, weights=[800, 800], optimal=[3050], name='G15'),
 Data(edge_index=[2, 39980], num_nodes=2000, weights=[2000, 2000], optimal=[13337], name='G24'),
 Data(edge_index=[2, 19980], num_nodes=1000, weights=[1000, 1000], optimal=[6654], name='G45'),
 Data(edge_index=[2, 23558], num_nodes=2000, weights=[2000, 2000], optimal=[7687], name='G38'),
 Data(edge_index=[2, 23570], num_nodes=2000, weights=[2000, 2000], optimal=[7689], name='G37'),
 Data(edge_index=[2, 82918], num_nodes=7000, weights=[7000, 7000], optimal=[26997], name='G63'),
 Data(edge_index=[2, 38352], num_nodes=800, weights=[800, 800], optimal=[11620], name='G2'),
 Data(edge_index=[2, 19980], num_nodes=1000, weights=[1000, 1000], optimal=[6657], name='G47'),
 Data(edge_index=[2, 11828], num_nodes=1000, weights=[1000, 1000], optimal=[3850], name='G53'),
 Data(edge_index=[2, 12000], num_nodes=3000, 